In [1]:
import ee
import os
import geemap
import json
import geopandas as gpd
import pprint as pp
from shapely.geometry import shape

ee.Authenticate()
ee.Initialize()

project_dir = 'projects/alpod-412314/assets/' 
region = 'YKF'
region_path = f'{region}_weekly'
folder = project_dir + region_path
#pp.pp(ee.data.listAssets(project_dir))
#pp.pp(ee.data.listAssets(project_dir + 'region_weekly/'))
#pp.pp(ee.data.listAssets(f'{project_dir}Lake_extractions/'))

# Read bufferd lake polygons
region_buffered_lakes = 'YKF' #!!! TUK_MRD is called TUK_MRD_AND for the lake regions
lakes_path = f'{project_dir}Lake_extractions/{region_buffered_lakes}_extraction'
lake_polygons = ee.FeatureCollection(lakes_path)

# Read weekly mosaics
weekly_mosaics = ee.data.listAssets(project_dir + 'region_weekly')
weekly_mosaics = weekly_mosaics['assets']

## 2.0 Select weekly mosaics for given timeperiods and ROI

In [2]:
target_years = list(range(2016, 2024))
target_years = [str(year) for year in target_years]

# June weeks
target_weeks = list(range(22, 27))
target_weeks = [str(week) for week in target_weeks]

max_observations = len(target_years) * len(target_weeks)
print(target_years, target_weeks)

target_imgs = []
target_imgs_footprints = []

for img in weekly_mosaics:
    img_id = img['id']
    temp_region = img_id.split('_')[-3]
    temp_region = temp_region.split('/')[-1]
    temp_year = img_id.split('_')[-2]
    temp_week = img_id.split('_')[-1]

    if temp_week in target_weeks and temp_year in target_years and temp_region == region:
        #print(img_id)
        image = ee.Image(img_id)
        image_info = image.getInfo()
        #pp.pp(image_info)
        image_coords = image_info['properties']['system:footprint']['coordinates']
        polygon_ee = ee.Geometry.Polygon(image_coords)

        target_imgs.append(image)
        target_imgs_footprints.append(polygon_ee)


p1 = target_imgs_footprints[0]
p2 = target_imgs_footprints[1]

combined_footprint = p1.union(p2)

for i in range(2, len(target_imgs_footprints)):
    p = target_imgs_footprints[i]
    combined_footprint = combined_footprint.union(p)

combined_footprint_info = combined_footprint.getInfo()
geojson_footprint = json.dumps(combined_footprint_info)
geometry = shape(json.loads(geojson_footprint))
roi_gdf = gpd.GeoDataFrame([{'geometry': geometry, 'roi': f'{region}'}], crs="EPSG:4326")
# os.chdir('/Users/jmaze/Documents/projects/altimetry_lakes_v3/')
# roi_gdf.to_file(f'./data/ew_rois/{region}_bbox.shp')


['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'] ['22', '23', '24', '25', '26']


## Conditional images

In [3]:
conditional_images = []

for img in target_imgs:

    # All observed pixels = 1
    obs_mask = img.unmask(0)
    obs_mask = obs_mask.rename('observed')
    #print(obs_mask.getInfo())
    
    # All water pixels = 2
    img_wtr = ee.Image.constant(0)
    img_wtr = img_wtr.where(img.select('water_occurance').eq(1), 2)
    img_wtr = img_wtr.rename('wtr_occurance')
    
    # All land pixels = 1
    img_land = ee.Image.constant(0)
    img_land = img_land.where(img.select('water_occurance').eq(0), 1)
    img_land = img_land.rename('land_occurance')
    
    # Lake shapes from original polygons 1=Lake, 0=Not a lake
    lakes_binary = lake_polygons.reduceToImage(
        properties=['n_lakes'],
        reducer=ee.Reducer.anyNonZero()
    ).neq(0)
    lakes_binary = lakes_binary.rename('buffered_lake')

    expr = """
    (wtr_observed == 2) ? 2 :
    (land_observed == 1) ? 1 :
    ((buffered_lake == 1) && (land_observed == 0) && (wtr_observed != 2)) ? 3 :
    0
    """
    conditional = ee.Image.constant(0)
    conditional = lakes_binary.expression(
        expr,
        {
            'wtr_observed': img_wtr.select('wtr_occurance'),
            'land_observed': img_land.select('land_occurance'),
            'buffered_lake': lakes_binary.select('buffered_lake'),
        }
    )
    
    conditional = conditional.rename('class')
    #print(conditional.getInfo())

    conditional_images.append(conditional)

In [4]:
collection = ee.ImageCollection(conditional_images)

def mask_and_set_to_one(image, class_value, footprint):
    image_updated = image.updateMask(image.eq(class_value)).multiply(0).add(1)
    return image_updated.clip(footprint)

# Create images for each class
land_images = collection.map(lambda img: mask_and_set_to_one(img.select('class'), 1, combined_footprint))
wtr_images = collection.map(lambda img: mask_and_set_to_one(img.select('class'), 2, combined_footprint))
inval_images = collection.map(lambda img: mask_and_set_to_one(img.select('class'), 3, combined_footprint))

# Sum the images
wtr_sum_img = wtr_images.sum()
land_sum_img = land_images.sum()
inval_observations_sum_img = inval_images.sum()

# Calc % water for observed images across each pixel
max_obs = len(target_imgs)
print(f'Highest possible observation count {max_obs}')

max_obs_image = ee.Image.constant(max_obs).clip(combined_footprint)
wtr_occurance_frac = wtr_sum_img.divide(max_obs_image.subtract(inval_observations_sum_img))

# Example: Print the result
print("Water sum image:", wtr_sum_img.getInfo())
print("Land sum image:", land_sum_img.getInfo())
print("Invalid observations sum image:", inval_observations_sum_img.getInfo())
pp.pp(wtr_occurance_frac.getInfo())



Highest possible observation count 38
Water sum image: {'type': 'Image', 'bands': [{'id': 'class', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -9.223372036854776e+18, 'max': 9.223372036854776e+18}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Land sum image: {'type': 'Image', 'bands': [{'id': 'class', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -9.223372036854776e+18, 'max': 9.223372036854776e+18}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Invalid observations sum image: {'type': 'Image', 'bands': [{'id': 'class', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -9.223372036854776e+18, 'max': 9.223372036854776e+18}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
{'type': 'Image',
 'bands': [{'id': 'class',
            'data_type': {'type': 'PixelType', 'precision': 'float'},
            'dimensions': [10, 3],
            'origin': [-151, 65],
            'crs': 'EPSG:4326',
            'crs_tran

In [ ]:
band = wtr_occurance_frac.select('class')

# coordinates = [
#     [-146.42929053, 66.34721564],
#     [-145.73500565, 66.32739946],
#     [-145.74722181, 66.15123003],
#     [-146.51124058, 66.15822638]
# ]

# # Create the polygon geometry
# t = ee.Geometry.Polygon([coordinates])

histogram = band.reduceRegion(
    reducer=ee.Reducer.fixedHistogram(min=0, max=1, steps=20),
    geometry=combined_footprint,
    scale=10,
    maxPixels=1e13,
)
result = histogram.getInfo()


# wtr_frac_viz = {
#     'bands': ['class'],
#     'min': 0,
#     'max': 1,
#     'palette': ['brown', 'green', 'blue']
# }


# Map = geemap.Map(center=(65, -135), zoom=4)
# Map.addLayer(wtr_occurance_frac, wtr_frac_viz, 'water frac')
# Map

In [23]:
pp.pp(result)import matplotlib.pyplot as plt

class_values, frequencies = zip(*result['class'])

# Create a bar plot
plt.figure(figsize=(12, 6))
plt.bar(class_values, frequencies, width=0.03, align='center')
plt.xticks(class_values, rotation=45)  # Rotate x-ticks for better visibility
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

{'class': [[0, 158463.58039215687],
           [0.05, 267614.39607843134],
           [0.1, 191810.09019607844],
           [0.15000000000000002, 80455.62745098039],
           [0.2, 69341.78823529412],
           [0.25, 61349.10196078431],
           [0.30000000000000004, 40174.52549019607],
           [0.35000000000000003, 30420.596078431372],
           [0.4, 38980.93725490196],
           [0.45, 33297.560784313726],
           [0.5, 44809.53725490196],
           [0.55, 42351.36862745098],
           [0.6000000000000001, 26024.141176470588],
           [0.65, 42943.407843137255],
           [0.7000000000000001, 26687.35294117647],
           [0.75, 36461.03137254902],
           [0.8, 45933.458823529414],
           [0.8500000000000001, 71779.74901960784],
           [0.9, 118972.91372549019],
           [0.9500000000000001, 82322.6]]}


Map(center=[65, -135], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [6]:
task = ee.batch.Export.image.toDrive(
    image=wtr_occurance_frac,
    description='test4real',
    folder='temp',
    fileNamePrefix='test4real',
    scale=10,
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [ ]:
obs_viz = {
    'bands': ['observed'],
    'min': 0,
    'max': 1,
    'palette': ['red', 'green']
}
lakes_viz = {
    'bands': ['buffered_lake'],
    'min': 0,
    'max': 1, 
    'palette': ['white', 'blue']
}
qk_viz = {
    'bands': ['wtr_occurance'],
    'min': 0,
    'max': 2, 
    'palette': ['white', 'blue']
}
qk_viz2 = {
    'bands': ['land_occurance'],
    'min': 0,
    'max': 1, 
    'palette': ['white', 'brown']
}

conditional_palette = ['grey', 'brown', 'blue', 'pink']
conditional_viz = {
    'band': ['class'],
    'min': 0,
    'max': 3,
    'palette': conditional_palette,
}

conditional_test = ee.Image(conditional_images[22])

# Create a map to display the results
Map = geemap.Map(center=(65, -135), zoom=4)
#Map.addLayer(obs_mask, obs_viz, 'Observation Mask')
Map.addLayer(conditional_test, conditional_viz, 'Conditional Test')
#Map.addLayer(lakes_binary, lakes_viz, 'Buffered Lakes')
#Map.addLayer(img_wtr, qk_viz, 'water')
#Map.addLayer(img_land, qk_viz2, 'land?')

Map